<a href="https://colab.research.google.com/github/andrejadikic/IgraZivotaPython/blob/main/Domaci1Zad2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import multiprocessing
import numpy as np
import threading
from threading import Thread
import random
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
from IPython.display import HTML

n = 5
# matrica trenutnog stanja
state = np.zeros((n, n)).astype(np.int8)
state[1][2]=1;
state[2][2]=1;
state[3][2]=1;
iter = 10
values = [0, 1]
# state_matrix = np.random.choice(values, n*n).reshape(n, n)

# queues pamti vrednosti za susede, to je red
queues = [multiprocessing.Queue(8) for _ in range(n ** 2)]
# pamti (step, curr_x, curr_y, cell_state)
cell_counter = multiprocessing.Queue(n ** 2)
# signalizira za svako polje u celiji da li moze da predje u sledecu iteraciju
next_iteration = [multiprocessing.Queue(1) for _ in range(n ** 2)]
conn1, conn2 = multiprocessing.Pipe()
# pamti sva stanja kako su se menjala kao istorija
steps = np.zeros(shape=(iter, n, n))


def isAlive(curr_val,alive_neigbors):
    if(alive_neigbors<2 or alive_neigbors>3):
      return 0
    elif((alive_neigbors==2 or alive_neigbors==3) and curr_val==1) or (curr_val==0 and alive_neigbors==3):
      return 1
    return curr_val

# vraca koliko ima komsija trenutni state[x][y]
def read(x,y):
    global n
    # na ivici
    if( ((x==0 or x==n-1) and (y>0 and y<n-1)) or ((y==0 or y==n-1) and (x>0 and x<n-1)) ): 
      return 5
    elif (x==0 or x==n-1) and (y==0 or y==n-1): #cosak
      return 3
    elif (x>0 and x<n-1) and (y<n-1 and y>0): #unutar
      return 8

#tid je kao ime procesa
#cell?state je u kom je stanju trenutni
def check_cells_around(tid, curr_x, curr_y, cell_state):
    cordinates = [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]

    for step in range(iter):
        alive_neigbors = 0
        # mesto u redu za x,y posto imamo red a ne matricu
        current_spot = curr_x * n + curr_y
        for x,y in cordinates:
            xx = (curr_x + x)
            yy = (curr_y + y) #koordinate komsije
            if(xx>=0 and xx<n and yy>=0 and yy<n): # da li je komsija u matrici
              neighbor_spot = xx * n + yy # mesto u redu za komsiju
              queues[neighbor_spot].put(cell_state) # komsiji dodajes u red stanje trenutnog polja da ga ne bi duplo posecivao
        for _ in range(read(curr_x,curr_y)): # imamo queues[current_spot] on u sebi ima broj vrednosti od komsija
            # taj quees ima razl velicine u zavisnosti dal je ivica,cosak,itd
            alive_neigbors += queues[current_spot].get()#broji zivi komsije prolazi kroz red i izracunava zbir tog reda
        #cell_state je sad novo updatovano stanje
        cell_state=isAlive(cell_state,alive_neigbors)
        # dodajemo u cell?counter red to novo stanje
        cell_counter.put((step, curr_x, curr_y, cell_state))
        # kad dobijes get od nove iteracije to je znak da pocinje sledeca iteracija
        next_iteration[current_spot].get()

#servis
def control():
    for _ in range(iter):
        for _ in range(n ** 2):
            message = cell_counter.get()  # kad je celija updated primamo poruku
            steps[message[0]][message[1]][message[2]] = message[3]
            # primili smo sve poruke posle fora

        for i in range(n ** 2):
            next_iteration[i].put(1) # saljemo signal svima da smo zavrsili
    conn1.send(steps)




def animate(steps):
    def init():
      im.set_data(steps[0])
      return [im]
    def animate(i):
      im.set_data(steps[i])
      return [im]
    im = plt.matshow(steps[0], interpolation='None', animated=True);
    anim = FuncAnimation(im.get_figure(), animate, init_func=init,
                    frames=len(steps), interval=500, blit=True, repeat=False);
    return anim
#controller je servis koji koristi funkciju control
controller = multiprocessing.Process(target=control)
#procesi kojima kontrolise servis
processes = []
for i in range(n ** 2):
    x = i // n # od indexa u redu izracunavamo x y koordinate
    y = i % n # pravimo procese za svaku celiju pocinjemo za tid od 2 zato sto je controler vec uzeo tid1
    processes.append(multiprocessing.Process(target=check_cells_around, args=(i + 2, x, y, state[x][y])))
#print(queues)
controller.start()
for p in processes:
    p.start()

for p in processes:
    p.join()
controller.join()

steps = conn2.recv()
print(steps)
conn1.close()
conn2.close()

[[[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 1. 1. 1. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 1. 1. 1. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 1. 1. 1. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 1. 1. 1. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 1. 1. 1. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]]]
